# Building an AI Book Research Assistant

This notebook provides step-by-step instructions for replicating what Andrew did in the second demo. Please follow along, and feel free to play with different variations too! At the end, there are optional instructions for building a more advanced book research assistant than Andrew showed. 

*Useful Resources*:

- **AISuite GitHub Repo**: [https://github.com/andrewyng/aisuite](https://github.com/andrewyng/aisuite)
- **Open Library API**:
  - [Open Library](https://openlibrary.org/developers/api) offers free and public Web APIs for accessing book and author catalog data (no API key is required)
  - See `openlibrary_api_docs.md` in this lesson's directory for more information

## Step 1: Use Open Library API's Documentation

- Open Jupyter chat by clicking on the chat bubble icon on the left sidebar of Jupyter Lab
- In the Chat interface, create a new chat by clicking `+Chat` (choose any name for the file)
- In the chat window, attach the Markdown file `openlibrary_api_docs.md` using the `@` symbol:
  - Type `@`, then select `file` from the autocomplete menu
  - You'll see a list of available files in the lesson's directory, choose `openlibrary_api_docs.md` 
- Use a prompt like this:
   > Write a code cell that uses Open Library's search endpoint to look up books based on a user's query and returns a response listing identified books, with these fields for each book: key, title, author_name, first_publish_year, isbn, cover_id, edition_count, language, subject. Use a variable called limit, set to 5, to specify the number of returned results. Print the results. Use the Open Library API documentation in the attached file.
- Transfer the generated code to the cell below and run it

In [1]:
import requests

# Define the search function
def search_books(query, limit=5):
    # Open Library Search API endpoint
    url = "https://openlibrary.org/search.json"
    
    # Parameters for the API request
    params = {
        'q': query,
        'limit': limit
    }
    
    # Make the GET request to the Open Library API
    response = requests.get(url, params=params)
    
    # Check if the response was successful
    if response.status_code == 200:
        data = response.json()
        books = data.get('docs', [])
        # Prepare the output
        results = []
        for book in books:
            results.append({
                'key': book.get('key'),
                'title': book.get('title'),
                'author_name': book.get('author_name', []),
                'first_publish_year': book.get('first_publish_year'),
                'isbn': book.get('isbn', []),
                'cover_id': book.get('cover_i'),
                'edition_count': book.get('edition_count'),
                'language': book.get('language', []),
                'subject': book.get('subject', [])
            })
        return results
    else:
        print("Error:", response.status_code, response.text)
        return []

# Example usage
user_query = input("Enter your search query: ")  # Replace with your search term
limit = 5
found_books = search_books(user_query, limit)

# Print the results
for book in found_books:
    print(f"Key: {book['key']}")
    print(f"Title: {book['title']}")
    print(f"Author: {', '.join(book['author_name'])}")
    print(f"First Publish Year: {book['first_publish_year']}")
    print(f"ISBN: {', '.join(book['isbn']) if book['isbn'] else 'N/A'}")
    print(f"Cover ID: {book['cover_id']}")
    print(f"Edition Count: {book['edition_count']}")
    print(f"Language: {', '.join(book['language']) if book['language'] else 'N/A'}")
    print(f"Subject: {', '.join(book['subject']) if book['subject'] else 'N/A'}")
    print("-" * 40)

Enter your search query:  non-fiction


Key: /works/OL29773770W
Title: We're All Puppets 2020
Author: Non Fiction
First Publish Year: 2019
ISBN: N/A
Cover ID: 13559199
Edition Count: 1
Language: eng
Subject: N/A
----------------------------------------
Key: /works/OL259399W
Title: Pocket Gde to Sketching Lb
Author: Non-Fiction
First Publish Year: 1986
ISBN: N/A
Cover ID: 10970390
Edition Count: 2
Language: N/A
Subject: N/A
----------------------------------------
Key: /works/OL32347798W
Title: Gem Guide to London
Author: Non-Fiction
First Publish Year: 1987
ISBN: N/A
Cover ID: None
Edition Count: 1
Language: N/A
Subject: N/A
----------------------------------------
Key: /works/OL85892W
Title: Dracula
Author: Bram Stoker
First Publish Year: 1897
ISBN: N/A
Cover ID: 12216503
Edition Count: 735
Language: alb, hun, spa, ger, ita, glg, fre, eng, tur, pol, dut, rus, chi, cat, por, gle
Subject: N/A
----------------------------------------
Key: /works/OL51831W
Title: πολιτεία
Author: Πλάτων
First Publish Year: 1554
ISBN: N/A
Cover I

## Step 2: Wrap the API Call in a Tool Function

- In the chat interface, use a prompt like this:
  > Wrap the code above in a function `search_books`, with good docstring.
  > 
  > - Input arguments: `query` and `limit` (with default value of 10)
  > - Output: JSON string with:
  >   - `num_found`: total results
  >   - `books`: list of dicts with title, authors, first_publish_year, key, edition_count

In [2]:
import requests
import json

def search_books(query, limit=10):
    """
    Search for books using the Open Library API.

    Parameters:
    - query (str): The search query for the book title, author, or subject.
    - limit (int): The number of results to return (default is 10).

    Returns:
    - str: A JSON string containing the total number of results and a list of books,
      where each book is represented as a dictionary with fields:
      - title: The title of the book.
      - authors: A list of authors for the book.
      - first_publish_year: The year the book was first published.
      - key: The Open Library key for the book.
      - edition_count: The number of editions available for the book.
    """
    # Open Library Search API endpoint
    url = "https://openlibrary.org/search.json"

    # Parameters for the API request
    params = {
        'q': query,
        'limit': limit
    }

    # Make the GET request to the Open Library API
    response = requests.get(url, params=params)

    # Check if the response was successful
    if response.status_code == 200:
        data = response.json()
        num_found = data.get('num_found', 0)
        books = []
        
        # Extract relevant information for each book
        for book in data.get('docs', []):
            books.append({
                'title': book.get('title'),
                'authors': book.get('author_name', []),
                'first_publish_year': book.get('first_publish_year'),
                'key': book.get('key'),
                'edition_count': book.get('edition_count')
            })
        
        # Construct the output dictionary
        output = {
            'num_found': num_found,
            'books': books
        }
        
        # Return the output as a JSON string
        return json.dumps(output)
    else:
        return json.dumps({"error": response.status_code, "message": response.text})

# Example usage
user_query = input("Enter your search query: ")  # Replace with your search term
result_json = search_books(user_query)

# Print the resulting JSON
print(result_json)

Enter your search query:  american revolution


{"num_found": 16380, "books": [{"title": "I Survived", "authors": ["Lauren Tarshis"], "first_publish_year": 2017, "key": "/works/OL17796132W", "edition_count": 3}, {"title": "Common Sense", "authors": ["Thomas Paine"], "first_publish_year": 1776, "key": "/works/OL60358W", "edition_count": 108}, {"title": "A Tale of Two Cities", "authors": ["Charles Dickens"], "first_publish_year": 1800, "key": "/works/OL8193465W", "edition_count": 2059}, {"title": "Rights of Man", "authors": ["Thomas Paine", "T. Paine", "Thomas Thomas Paine"], "first_publish_year": 1791, "key": "/works/OL60359W", "edition_count": 276}, {"title": "The Iron Heel", "authors": ["Jack London"], "first_publish_year": 1907, "key": "/works/OL74502W", "edition_count": 594}, {"title": "Hard Times", "authors": ["Charles Dickens"], "first_publish_year": 1854, "key": "/works/OL8193387W", "edition_count": 1029}, {"title": "Study Guide", "authors": ["SuperSummary"], "first_publish_year": 2019, "key": "/works/OL21885963W", "edition_co

- Test the function by uncommenting the code below:

In [3]:
results = search_books("science fiction")

In [4]:
print(results)

{"num_found": 99816, "books": [{"title": "The Science Fiction Hall of Fame -- Volume One", "authors": ["Robert Silverberg", "Robert A. Heinlein", "Arthur C. Clarke", "Isaac Asimov", "Ray Bradbury"], "first_publish_year": 1970, "key": "/works/OL1960472W", "edition_count": 31}, {"title": "Foundation", "authors": ["Isaac Asimov"], "first_publish_year": 1951, "key": "/works/OL46125W", "edition_count": 97}, {"title": "Stranger in a Strange Land", "authors": ["Robert A. Heinlein"], "first_publish_year": 1961, "key": "/works/OL59688W", "edition_count": 80}, {"title": "The Last Man", "authors": ["Mary Shelley"], "first_publish_year": 1826, "key": "/works/OL450124W", "edition_count": 383}, {"title": "The Time Machine", "authors": ["H. G. Wells"], "first_publish_year": 1895, "key": "/works/OL52267W", "edition_count": 1154}, {"title": "Dune", "authors": ["Frank Herbert"], "first_publish_year": 1965, "key": "/works/OL893415W", "edition_count": 121}, {"title": "The Invisible Man", "authors": ["H. G

## Step 3: Build a Chat Handler with Automatic Function Calling

- Drag and drop the raw cell below into the chat window and use a prompt like this:
  > Use this aisuite example to create a simple chat handler function `chat_with_tools`:
  > - input: `user_message`
  > - output: `response.choices[0].message.content`

In [7]:
import aisuite as ai
import json
import requests

# The previously defined search_books function
def search_books(query: str, limit: int = 10) -> str:
    """
    Search for books using the Open Library API.

    Parameters:
    - query (str): The search query for the book title, author, or subject.
    - limit (int): The number of results to return (default is 10).

    Returns:
    - str: A JSON string containing the total number of results and a list of books,
      where each book is represented as a dictionary with fields:
      - title: The title of the book.
      - authors: A list of authors for the book.
      - first_publish_year: The year the book was first published.
      - key: The Open Library key for the book.
      - edition_count: The number of editions available for the book.
    """
    url = "https://openlibrary.org/search.json"
    params = {'q': query, 'limit': limit}
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        num_found = data.get('num_found', 0)
        books = []

        for book in data.get('docs', []):
            books.append({
                'title': book.get('title'),
                'authors': book.get('author_name', []),
                'first_publish_year': book.get('first_publish_year'),
                'key': book.get('key'),
                'edition_count': book.get('edition_count')
            })

        output = {
            'num_found': num_found,
            'books': books
        }
        return json.dumps(output)
    else:
        return json.dumps({"error": response.status_code, "message": response.text})

# Chat handler function
def chat_with_tools(user_message: str) -> str:
    """
    Handle user messages and respond with suggestions from the search_books tool.

    Parameters:
    - user_message (str): The message inputted by the user.

    Returns:
    - str: The response content from the chat model.
    """
    client = ai.Client()
    
    # Prepare the message for the chat request
    messages = [{
        "role": "user",
        "content": user_message
    }]
    
    # Make a chat completion request using the tools
    response = client.chat.completions.create(
        model="openai:gpt-4.1-mini",
        messages=messages,
        tools=[search_books],
        max_turns=3  # Maximum number of back-and-forth tool calls
    )
    
    # Return the content of the response
    return response.choices[0].message.content

# Example usage
user_query = "Suggest books about Science Fiction"
response_content = chat_with_tools(user_query)

# Print the response from the chat
print(response_content)

Here are some notable Science Fiction books you might like:

1. The Science Fiction Hall of Fame -- Volume One by Robert Silverberg, Robert A. Heinlein, Arthur C. Clarke, Isaac Asimov, Ray Bradbury (first published in 1970)
2. Foundation by Isaac Asimov (first published in 1951)
3. Stranger in a Strange Land by Robert A. Heinlein (first published in 1961)
4. The Last Man by Mary Shelley (first published in 1826)
5. The Time Machine by H. G. Wells (first published in 1895)

Would you like more information on any of these books?


## Step 4: Test the Chat Handler

- Run the following code cell to test `chat_with_tools`
- Feel free to search for different types of books
- Optional: If your generated function `search_books` does not contain printing statements, you can update the function as Andrew showed in the video:
  - prompt used: modify the search_books function to add a printing statement showing when it is called with basic status messages.

In [8]:
print("="*60)
print("EXAMPLE: Science Function book search")
print("="*60)

user_query = "Search for books about science fiction"
print(f"\n👤 User: {user_query}")

response = chat_with_tools(user_query)

print(f"\n🤖 Assistant: {response}")

EXAMPLE: Science Function book search

👤 User: Search for books about science fiction

🤖 Assistant: Here are some books about science fiction:

1. The Science Fiction Hall of Fame -- Volume One by Robert Silverberg, Robert A. Heinlein, Arthur C. Clarke, Isaac Asimov, Ray Bradbury (First published in 1970)
2. Foundation by Isaac Asimov (First published in 1951)
3. Stranger in a Strange Land by Robert A. Heinlein (First published in 1961)
4. The Last Man by Mary Shelley (First published in 1826)
5. The Time Machine by H. G. Wells (First published in 1895)
6. Dune by Frank Herbert (First published in 1965)
7. The Invisible Man by H. G. Wells
8. Мы by Евгений Иванович Замятин and others (First published in 1924)
9. Wild Seed by Octavia E. Butler (First published in 1980)
10. Great Sf Heinlein Bxs by Robert A. Heinlein (First published in 1980)

If you want details or more books, just let me know!


## Step 5: Look up Details on Books (optional) 

You can repeat steps 1-3 to define a second tool for the book research assistant, so it can handle more complex book queries.
For example, you can use the `works` endpoint to retrieve more detailed information about a particular book given its work id. For more details, check `openlibrary_api_docs.md`.

Here are steps with suggested prompts:

1. In the chat interface, attach the file containing the API documentation of Open Library (`openlibrary_api_docs.md`) and use a prompt like this:
   > Write a code cell that uses Open Library's works endpoint to retrieve information about a book, based on the work's key that has this format: "/works/{work_id}". The result should include: title, description, subjects (limit to 10), first_sentence, covers (limit to 3). Print the returned results. Use the documentation for Open Library's API provided above.
2. In the chat interface, create the function tool using a prompt like this:
   > Wrap the code above in a function `get_book_details`:
   > - Function argument: `work_key` in this format: `/works/{work_id}`
   > - Output: JSON string with: title, description, subjects (limit to 10), first_sentence, covers (limit to 3)
   >  - Write good docstring
3. Update the `chat_with_tools` function to include the second tool as follows:
   
   ``` python
   response = client.chat.completions.create(
    model="openai:gpt-4.1-mini",
    messages=messages,
    tools=[search_books, get_book_details],
    max_turns=3  # Maximum number of back-and-forth tool calls
   )
   ```
   
4. Try this query:
   ```python
   user_query = "Search for books with 'Linear Algebra' in the title and tell me about the one with the most editions"
   ```

In [9]:
import requests

def get_book_info(work_key):
    """
    Retrieve detailed information about a book from the Open Library Works API.

    Parameters:
    - work_key (str): The key for the specific work in the format "/works/{work_id}".

    Returns:
    - dict: A dictionary containing the title, description, subjects, first sentence, and covers.
    """
    # Construct the URL using the provided work key
    url = f'https://openlibrary.org{work_key}.json'

    # Make the GET request to the Open Library Works API
    response = requests.get(url)

    # Check if the response was successful
    if response.status_code == 200:
        data = response.json()

        # Extract the required information
        title = data.get('title', 'Unknown Title')
        description = data.get('description', 'No description available')
        subjects = data.get('subjects', [])[:10]  # Limit to 10 subjects
        first_sentence = data.get('first_sentence', ['No first sentence available'])[0]  # Get first sentence
        covers = data.get('covers', [])[:3]  # Limit to 3 cover IDs

        # Prepare the result
        result = {
            'title': title,
            'description': description,
            'subjects': subjects,
            'first_sentence': first_sentence,
            'covers': covers
        }
        return result
    else:
        print(f"Error: Unable to retrieve information for {work_key}, Status Code: {response.status_code}")
        return None

# Example usage
work_key = "/works/OL14933374W"  # Replace with the desired work key
book_info = get_book_info(work_key)

# Print the returned results
if book_info:
    print("Title:", book_info['title'])
    print("Description:", book_info['description'])
    print("Subjects:", book_info['subjects'])
    print("First Sentence:", book_info['first_sentence'])
    print("Cover IDs:", book_info['covers'])

Title: An English forest
Description: No description available
Subjects: []
First Sentence: No first sentence available
Cover IDs: [5688862, 7189461]


In [10]:
import requests
import json

def get_book_details(work_key):
    """
    Retrieve detailed information about a book from the Open Library Works API.

    Parameters:
    - work_key (str): The key for the specific work in the format "/works/{work_id}".

    Returns:
    - str: A JSON string containing the title, description, subjects (limited to 10),
           first sentence, and covers (limited to 3). If an error occurs, a JSON
           string with an error message is returned.
    """
    # Construct the URL using the provided work key
    url = f'https://openlibrary.org{work_key}.json'

    # Make the GET request to the Open Library Works API
    response = requests.get(url)

    # Check if the response was successful
    if response.status_code == 200:
        data = response.json()

        # Extract the required information
        title = data.get('title', 'Unknown Title')
        description = data.get('description', 'No description available')
        subjects = data.get('subjects', [])[:10]  # Limit to 10 subjects
        first_sentence = data.get('first_sentence', ['No first sentence available'])[0]  # Get first sentence
        covers = data.get('covers', [])[:3]  # Limit to 3 cover IDs

        # Prepare the result as a dictionary
        result = {
            'title': title,
            'description': description,
            'subjects': subjects,
            'first_sentence': first_sentence,
            'covers': covers
        }
        
        # Return the result as a JSON string
        return json.dumps(result)
    else:
        # Return error details as a JSON string
        error_result = {
            "error": "Unable to retrieve information",
            "status_code": response.status_code,
            "message": response.text
        }
        return json.dumps(error_result)

# Example usage
work_key = "/works/OL14933374W"  # Replace with the desired work key
book_details = get_book_details(work_key)

# Print the returned JSON string
print(book_details)

{"title": "An English forest", "description": "No description available", "subjects": [], "first_sentence": "No first sentence available", "covers": [5688862, 7189461]}
